In [7]:
import torch
import torch.nn as nn
import torch.utils.data as data

import torch.optim as optim

import torchvision
from torchvision import datasets as dsets
from torchvision import transforms

from matplotlib.pyplot import imshow
%matplotlib inline

import visdom
import math
import vgg
import numpy as np

In [8]:
vis = visdom.Visdom()
vis.close(env="main")

''

In [9]:
def plt_tracker(loss_plot, loss_value, num):
    vis.line(X=np.stack(np.arange(num, num+1)),
             Y=np.stack([loss_value]),
             win=loss_plot,
             update='append'
             )

In [10]:
if (torch.cuda.is_available() == 1):
    print("cuda is available")
    device = 'cuda'
else:
    device = 'cpu'

cuda is available


In [11]:
transform = transforms.Compose([
    #transforms.Resize(256),
    transforms.RandomCrop(56),
    transforms.ToTensor(),]
    #transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))]
)

In [24]:
try:
    train = dsets.ImageFolder(root='./tiny-imagenet-200/train',transform=transform)
    print("Dataset is alreay downlaoded")
except:
    !./download.sh
    train = dsets.ImageFolder(root='./tiny-imagenet-200/train',transform=transform)
    
        
validation = dsets.ImageFolder(root='./tiny-imagenet-200/val',transform=transform)
test = dsets.ImageFolder(root='./tiny-imagenet-200/test',transform=transform)

print(train)
print(validation)
print(test)


check_img=train.__getitem__(0)
ToPIL=transforms.ToPILImage()
check_img = ToPIL(check_img[0])
print(check_img)

imshow(check_img)

./download.sh: 1: ./download.sh: [-f: not found
Download Tiny-ImageNet Dataset
--2019-02-09 02:14:45--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu... 171.64.68.10
Connecting to cs231n.stanford.edu|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z  23%[===>                ]  55.28M  11.9MB/s    eta 32s    ^C


FileNotFoundError: [Errno 2] No such file or directory: './tiny-imagenet-200/train'

In [7]:
train_loader=data.DataLoader(train, batch_size=64, shuffle=True, num_workers=2)
var_loader=data.DataLoader(validation, batch_size=64, shuffle=False, num_workers=2)
test_loader=data.DataLoader(test, batch_size=64, shuffle=False, num_workers=2)

In [8]:
conv = vgg.make_layers(vgg.cfg['tiny-imagenet'],True)

In [9]:
class tiny_VGG(nn.Module):
    def __init__(self,conv_layer,num_of_class=200, init_weights=True):
        super(tiny_VGG, self).__init__()
        self.conv = conv_layer
        self.fc = nn.Sequential(
            nn.Linear(7*7*512, 1024),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(1024, 1024),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(1024, num_of_class))
        
        if init_weights:
            self._initialize_weights()
        
    def forward(self,x):
        x = self.conv(x)
        #print(x.shape)
        x=x.view(x.shape[0], -1)
        
        x=self.fc(x)
        return x
        
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()

In [10]:
CNN = tiny_VGG(conv)
CNN=CNN.to(device)
print(CNN)

tiny_VGG(
  (conv): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256, kernel_size=(3, 3

In [11]:
test_input = torch.Tensor(3,3,56,56).to(device)

out = CNN(test_input)
print(out.shape)

torch.Size([3, 200])


In [12]:
optimizer = optim.SGD(CNN.parameters(),lr=0.01,momentum=0.9)
loss_function = nn.CrossEntropyLoss()

lr_sche = optim.lr_scheduler.StepLR(optimizer,step_size=9,gamma=0.5)

In [13]:
loss_plot = vis.line(Y=torch.Tensor(1).zero_())

In [14]:
epochs = 30
print("length of train_loader : {}".format(len(train_loader)))
for epoch in range(epochs):
    running_loss = 0.0
    check_loss = 0.0
    lr_sche.step()
    for num, data in enumerate(train_loader):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        out = CNN(inputs)
        loss = loss_function(out, labels)
        loss.backward()
        
        optimizer.step()
        running_loss += loss.item()
        check_loss += loss.item()
        
        if (num % 100 == 1) and (num != 1):
            check_loss = check_loss/100
            print("loss : {}".format(check_loss))
            plt_tracker(loss_plot,(check_loss), epoch *len(train_loader) + num)
            check_loss = 0  
        
        if num % len(train_loader) == (len(train_loader) - 1):
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, num + 1, running_loss / len(train_loader)))
            running_loss = 0.0

length of train_loader : 1563
loss : 5.409017748832703
loss : 5.218698024749756
loss : 5.099050221443176
loss : 5.018397035598755
loss : 4.929066796302795
loss : 4.8695987653732296
loss : 4.796232261657715
loss : 4.70706612110138
loss : 4.646880164146423
loss : 4.609096693992615
loss : 4.54817684173584
loss : 4.5146562242507935
loss : 4.459525709152222
loss : 4.4313158655166625
loss : 4.369911780357361
[1,  1563] loss: 4.755
loss : 4.429037697315216
loss : 4.294278047084808
loss : 4.305355687141418
loss : 4.234861006736756
loss : 4.1734132170677185
loss : 4.1669985914230345
loss : 4.183407974243164
loss : 4.181097640991211
loss : 4.153704938888549
loss : 4.149314892292023
loss : 4.1225672173500065
loss : 4.110350966453552
loss : 4.061641840934754
loss : 4.0740625238418575
loss : 4.0653898143768314
[2,  1563] loss: 4.168
loss : 4.030554580688476
loss : 3.9553109407424927
loss : 3.941073729991913
loss : 3.9217844414711
loss : 3.9177987790107727
loss : 3.878767945766449
loss : 3.890950417

loss : 1.1064160215854644
loss : 1.1004332780838013
loss : 1.1044199436903
loss : 1.0684518480300904
loss : 1.0907342451810838
loss : 1.1170688146352767
loss : 1.0965173244476318
loss : 1.1147806465625762
loss : 1.123497303724289
loss : 1.121470085978508
loss : 1.1308887380361556
loss : 1.117449722290039
loss : 1.1100248301029205
[21,  1563] loss: 1.104
loss : 1.0131647592782975
loss : 0.9865172308683395
loss : 1.023768383860588
loss : 1.0022771489620208
loss : 0.9950504273176193
loss : 1.0246196043491365
loss : 1.0573646157979966
loss : 1.0510306859016418
loss : 1.0036785727739335
loss : 1.048923870921135
loss : 1.0794554680585862
loss : 1.0560987454652786
loss : 1.0935564124584198
loss : 1.020211796760559
loss : 1.0688773781061172
[22,  1563] loss: 1.035
loss : 0.9694001144170761
loss : 0.9348641979694366
loss : 0.9370287495851517
loss : 0.9690099650621414
loss : 0.934783935546875
loss : 0.9703447839617729
loss : 0.9908861607313156
loss : 0.9776946365833282
loss : 0.9965801638364792


In [15]:
torch.save(CNN.state_dict(),"./model/tiny_imagenet_vgg.pth")

In [16]:
load_conv = vgg.make_layers(vgg.cfg['tiny-imagenet'],True)
load_CNN = tiny_VGG(load_conv)

In [17]:
load_CNN.load_state_dict(torch.load('./model/tiny_imagenet_vgg.pth',map_location = device))
load_CNN = load_CNN.to(device)

In [22]:
print(load_CNN.conv[0].weight.data[0][0])
print(load_CNN.conv[0].bias.data[0])

tensor([[ 0.1941,  0.0323, -0.3319],
        [ 0.5728,  0.1229, -0.3863],
        [ 0.3556,  0.2330,  0.0028]], device='cuda:0')
tensor(-1.0122e-06, device='cuda:0')


In [23]:
print(CNN.conv[0].weight.data[0][0])
print(CNN.conv[0].bias.data[0])

tensor([[ 0.1941,  0.0323, -0.3319],
        [ 0.5728,  0.1229, -0.3863],
        [ 0.3556,  0.2330,  0.0028]], device='cuda:0')
tensor(-1.0122e-06, device='cuda:0')


In [24]:
dir(loss_plot)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmod__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'capitalize',
 'casefold',
 'center',
 'count',
 'encode',
 'endswith',
 'expandtabs',
 'find',
 'format',
 'format_map',
 'index',
 'isalnum',
 'isalpha',
 'isascii',
 'isdecimal',
 'isdigit',
 'isidentifier',
 'islower',
 'isnumeric',
 'isprintable',
 'isspace',
 'istitle',
 'isupper',
 'join',
 'ljust',
 'lower',
 'lstrip',
 'maketrans',
 'partition',
 'replace',
 'rfind',
 'rindex',
 'rjust',
 'rpartition',
 'rsplit',
 'rstrip',
 'split',
 'splitlines',
 'startswith',
 'strip',
 'swapcase',
 'title',
 'translate',
 'upper',


ModuleNotFoundError: No module named 'wget'

NameError: name 'wget' is not defined